# 缩放点积自注意力机制 (Scaled Dot-Product Self-Attention)

**SOTA 教育标准实现** | Transformer 架构核心组件

---

## 1. 理论基础与数学推导

### 1.1 核心公式

缩放点积注意力的标准形式：

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

其中：
- $Q \in \mathbb{R}^{n \times d_k}$：Query 矩阵
- $K \in \mathbb{R}^{m \times d_k}$：Key 矩阵  
- $V \in \mathbb{R}^{m \times d_v}$：Value 矩阵
- $d_k$：Key/Query 的维度

### 1.2 缩放因子的数学证明 ⭐

**定理**：假设 $Q, K$ 的元素独立同分布，均值为 0，方差为 1，则 $Q \cdot K^T$ 的方差为 $d_k$。

**证明**：

设 $q = (q_1, q_2, \ldots, q_{d_k})$ 和 $k = (k_1, k_2, \ldots, k_{d_k})$ 分别为 $Q$ 和 $K$ 的某一行向量。

点积定义为：
$$z = q \cdot k = \sum_{i=1}^{d_k} q_i k_i$$

**Step 1: 计算期望**

由于 $q_i$ 和 $k_i$ 独立且 $\mathbb{E}[q_i] = \mathbb{E}[k_i] = 0$：

$$\mathbb{E}[z] = \mathbb{E}\left[\sum_{i=1}^{d_k} q_i k_i\right] = \sum_{i=1}^{d_k} \mathbb{E}[q_i] \cdot \mathbb{E}[k_i] = 0$$

**Step 2: 计算方差**

$$\text{Var}(z) = \text{Var}\left(\sum_{i=1}^{d_k} q_i k_i\right)$$

由于各项独立：
$$= \sum_{i=1}^{d_k} \text{Var}(q_i k_i)$$

对于独立随机变量的乘积，有：
$$\text{Var}(q_i k_i) = \mathbb{E}[q_i^2 k_i^2] - (\mathbb{E}[q_i k_i])^2 = \mathbb{E}[q_i^2] \cdot \mathbb{E}[k_i^2] - 0 = 1 \cdot 1 = 1$$

因此：
$$\boxed{\text{Var}(z) = \sum_{i=1}^{d_k} 1 = d_k}$$

**直觉解释**：

当 $d_k = 512$ 时，点积的标准差为 $\sqrt{512} \approx 22.6$。如此大的数值会使 Softmax 输出趋近于 one-hot 分布，导致：
1. **梯度消失**：Softmax 在极端值处梯度接近 0
2. **注意力坍缩**：模型只关注单一位置，丧失全局建模能力

除以 $\sqrt{d_k}$ 后，方差重新归一化为 1，Softmax 保持在健康的梯度区间。

---

## 2. 代码实现 (SOTA 标准)

In [ ]:
from __future__ import annotations

import math
from dataclasses import dataclass
from typing import Optional, Tuple

import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

In [ ]:
@dataclass
class AttentionConfig:
    """注意力机制配置类。

    使用 dataclass 管理所有超参数，避免魔术数字散落在代码中。

    Attributes:
        d_k: Key/Query 维度，用于计算缩放因子
        d_v: Value 维度，决定输出维度
        dropout: Dropout 概率，用于注意力权重正则化
        mask_value: 掩码填充值，使用 -1e9 而非 -inf 保证数值稳定性
    """

    d_k: int = 64
    d_v: int = 64
    dropout: float = 0.0
    mask_value: float = -1e9  # 数值稳定性：避免使用 -inf

In [ ]:
class ScaledDotProductAttention(nn.Module):
    """缩放点积注意力机制 (Scaled Dot-Product Attention)。

    核心思想 (Core Idea):
        通过 Query 与 Key 的点积计算相似度，经 Softmax 归一化后对 Value 加权求和。
        缩放因子 1/sqrt(d_k) 防止高维点积导致的梯度消失。

    数学原理 (Mathematical Theory):
        Attention(Q, K, V) = softmax(QK^T / sqrt(d_k)) * V

        其中 einsum 约定:
        - 'b i d, b j d -> b i j': Q @ K^T (注意力分数)
        - 'b i j, b j d -> b i d': weights @ V (加权求和)

    复杂度 (Complexity):
        - 时间: O(n^2 * d) 其中 n 为序列长度
        - 空间: O(n^2) 用于存储注意力矩阵
    """

    def __init__(self, config: AttentionConfig) -> None:
        super().__init__()
        self.config = config
        self.scale: float = 1.0 / math.sqrt(config.d_k)
        self.dropout = nn.Dropout(p=config.dropout)

    def forward(
        self,
        q: Tensor,  # shape: (batch, seq_q, d_k)
        k: Tensor,  # shape: (batch, seq_k, d_k)
        v: Tensor,  # shape: (batch, seq_k, d_v)
        mask: Optional[Tensor] = None,  # shape: (batch, seq_q, seq_k)
    ) -> Tuple[Tensor, Tensor]:
        """前向传播。

        Args:
            q: Query 张量
            k: Key 张量
            v: Value 张量
            mask: 掩码张量，True/1 的位置将被屏蔽

        Returns:
            output: 注意力输出 (batch, seq_q, d_v)
            attention_weights: 注意力权重 (batch, seq_q, seq_k)
        """
        # Step 1: 计算注意力分数 (使用 einsum 提升可读性)
        # einsum: 'b i d, b j d -> b i j' 表示 batch 矩阵乘法 Q @ K^T
        scores: Tensor = torch.einsum("b i d, b j d -> b i j", q, k)

        # Step 2: 缩放 (防止方差过大导致 Softmax 饱和)
        scores = scores * self.scale

        # Step 3: 应用掩码 (使用 -1e9 而非 -inf 保证数值稳定)
        if mask is not None:
            scores = scores.masked_fill(mask.bool(), self.config.mask_value)

        # Step 4: Softmax 归一化
        attention_weights: Tensor = F.softmax(scores, dim=-1)

        # Step 5: Dropout 正则化
        attention_weights = self.dropout(attention_weights)

        # Step 6: 加权求和
        # einsum: 'b i j, b j d -> b i d' 表示 weights @ V
        output: Tensor = torch.einsum("b i j, b j d -> b i d", attention_weights, v)

        return output, attention_weights

---

## 3. 验证测试

In [ ]:
def test_forward_pass() -> None:
    """验证前向传播的输出形状正确性。"""
    config = AttentionConfig(d_k=64, d_v=64)
    attention = ScaledDotProductAttention(config)

    batch_size, seq_len = 2, 10
    q = torch.randn(batch_size, seq_len, config.d_k)
    k = torch.randn(batch_size, seq_len, config.d_k)
    v = torch.randn(batch_size, seq_len, config.d_v)

    output, weights = attention(q, k, v)

    assert output.shape == (batch_size, seq_len, config.d_v), f"输出形状错误: {output.shape}"
    assert weights.shape == (batch_size, seq_len, seq_len), f"权重形状错误: {weights.shape}"

    # 验证 Softmax 归一化 (每行和为 1)
    row_sums = weights.sum(dim=-1)
    assert torch.allclose(row_sums, torch.ones_like(row_sums), atol=1e-5), "Softmax 归一化失败"

    print("✓ test_forward_pass 通过")


test_forward_pass()

In [ ]:
def test_variance_scaling() -> None:
    """验证缩放因子确实将方差归一化。"""
    torch.manual_seed(42)
    d_k = 512
    n_samples = 10000

    # 生成标准正态分布的 Q, K
    q = torch.randn(n_samples, d_k)
    k = torch.randn(n_samples, d_k)

    # 未缩放的点积
    unscaled = torch.einsum("i d, i d -> i", q, k)
    print(f"未缩放点积 - 均值: {unscaled.mean():.4f}, 方差: {unscaled.var():.4f} (理论值: {d_k})")

    # 缩放后的点积
    scaled = unscaled / math.sqrt(d_k)
    print(f"缩放后点积 - 均值: {scaled.mean():.4f}, 方差: {scaled.var():.4f} (理论值: 1.0)")

    assert abs(unscaled.var().item() - d_k) < 50, "未缩放方差偏离理论值过大"
    assert abs(scaled.var().item() - 1.0) < 0.1, "缩放后方差未归一化"

    print("✓ test_variance_scaling 通过")


test_variance_scaling()

---

## 4. 高标验证：真实句子 Demo

In [ ]:
def visualize_attention_on_sentence(
    sentence: str = "Thinking strictly is hard",
    d_k: int = 64,
    seed: int = 42,
) -> None:
    """在真实句子上可视化注意力权重。

    Args:
        sentence: 输入句子
        d_k: 嵌入维度
        seed: 随机种子
    """
    torch.manual_seed(seed)

    tokens = sentence.split()
    seq_len = len(tokens)

    # 模拟词嵌入 (实际应用中使用预训练嵌入)
    embeddings = torch.randn(1, seq_len, d_k)

    # 线性投影生成 Q, K, V
    W_q = nn.Linear(d_k, d_k, bias=False)
    W_k = nn.Linear(d_k, d_k, bias=False)
    W_v = nn.Linear(d_k, d_k, bias=False)

    q = W_q(embeddings)
    k = W_k(embeddings)
    v = W_v(embeddings)

    # 计算注意力
    config = AttentionConfig(d_k=d_k)
    attention = ScaledDotProductAttention(config)
    _, weights = attention(q, k, v)

    # 绘制热力图
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        weights[0].detach().numpy(),
        xticklabels=tokens,
        yticklabels=tokens,
        annot=True,
        fmt=".3f",
        cmap="Blues",
        cbar_kws={"label": "Attention Weight"},
        square=True,
    )
    plt.xlabel("Key (被关注的词)", fontsize=12)
    plt.ylabel("Query (发起查询的词)", fontsize=12)
    plt.title(f'Self-Attention Weights: "{sentence}"', fontsize=14)
    plt.tight_layout()
    plt.show()

    # 分析结果
    print("\n注意力分析:")
    for i, token in enumerate(tokens):
        top_k = 2
        top_indices = weights[0, i].topk(top_k).indices.tolist()
        top_tokens = [tokens[j] for j in top_indices]
        print(f"  '{token}' 最关注: {top_tokens}")


visualize_attention_on_sentence()

### 热力图解读

- **行 (Y轴)**：Query 位置，表示"谁在发起查询"
- **列 (X轴)**：Key 位置，表示"被关注的对象"
- **颜色深浅**：注意力权重大小，越深表示关注度越高
- **每行之和 = 1**：Softmax 保证归一化

在训练后的模型中，我们期望看到：
- "Thinking" 可能关注 "hard" (语义关联)
- "strictly" 可能关注 "Thinking" (修饰关系)

---

## 5. 因果掩码 (Causal Mask) 可视化

In [ ]:
def visualize_causal_mask(seq_len: int = 5) -> Tensor:
    """可视化因果掩码的工作原理。

    因果掩码阻止位置 i 关注位置 j > i，防止信息泄漏。
    """
    # 创建因果掩码 (上三角为 True)
    causal_mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # 左图：掩码矩阵
    ax1 = axes[0]
    mask_visual = causal_mask.float().numpy()
    sns.heatmap(
        mask_visual,
        ax=ax1,
        annot=True,
        fmt=".0f",
        cmap="Reds",
        cbar_kws={"label": "Masked (1) / Visible (0)"},
        square=True,
        xticklabels=[f"t={i}" for i in range(seq_len)],
        yticklabels=[f"t={i}" for i in range(seq_len)],
    )
    ax1.set_xlabel("Key 位置 (被关注)")
    ax1.set_ylabel("Query 位置 (发起查询)")
    ax1.set_title("因果掩码矩阵\n(1=屏蔽未来, 0=可见)")

    # 右图：信息流向示意
    ax2 = axes[1]
    allowed = (~causal_mask).float().numpy()
    sns.heatmap(
        allowed,
        ax=ax2,
        annot=True,
        fmt=".0f",
        cmap="Greens",
        cbar_kws={"label": "Allowed (1) / Blocked (0)"},
        square=True,
        xticklabels=[f"t={i}" for i in range(seq_len)],
        yticklabels=[f"t={i}" for i in range(seq_len)],
    )
    ax2.set_xlabel("Key 位置")
    ax2.set_ylabel("Query 位置")
    ax2.set_title("允许的注意力连接\n(下三角 + 对角线)")

    plt.tight_layout()
    plt.show()

    print("\n信息流解释:")
    print("  - 位置 0 只能看到自己")
    print("  - 位置 1 可以看到位置 0, 1")
    print("  - 位置 i 可以看到位置 0, 1, ..., i")
    print("  - 这防止了自回归生成时的信息泄漏")

    return causal_mask


causal_mask = visualize_causal_mask()

In [ ]:
def demo_masked_attention(sentence: str = "The cat sat on") -> None:
    """演示带因果掩码的注意力。"""
    torch.manual_seed(42)

    tokens = sentence.split()
    seq_len = len(tokens)
    d_k = 64

    # 创建输入
    q = torch.randn(1, seq_len, d_k)
    k = torch.randn(1, seq_len, d_k)
    v = torch.randn(1, seq_len, d_k)

    # 因果掩码
    mask = torch.triu(torch.ones(1, seq_len, seq_len), diagonal=1)

    # 计算注意力
    config = AttentionConfig(d_k=d_k)
    attention = ScaledDotProductAttention(config)
    _, weights = attention(q, k, v, mask=mask)

    # 可视化
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        weights[0].detach().numpy(),
        xticklabels=tokens,
        yticklabels=tokens,
        annot=True,
        fmt=".3f",
        cmap="Oranges",
        cbar_kws={"label": "Attention Weight"},
        square=True,
    )
    plt.xlabel("Key (被关注的词)")
    plt.ylabel("Query (发起查询的词)")
    plt.title(f'Masked Self-Attention: "{sentence}"\n(上三角被屏蔽)')
    plt.tight_layout()
    plt.show()

    print("\n观察: 上三角区域权重为 0，每个词只能关注自己和之前的词")


demo_masked_attention()

---

## 6. 多头注意力机制 (Multi-Head Attention) ⭐

### 6.1 核心思想

**问题**: 单头注意力只能学习一种"关注模式"。

**解决方案**: 并行运行多个注意力头，每个头学习不同的关注模式。

$$\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, ..., \text{head}_h)W^O$$

其中 $\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)$

**直觉**: 不同的头可以关注不同的语义关系：
- Head 1: 语法依赖 (主谓关系)
- Head 2: 指代消解 (代词指向)
- Head 3: 位置关系 (相邻词)

In [ ]:
class MultiHeadAttention(nn.Module):
    """多头注意力机制 (Multi-Head Attention)。

    核心思想:
        将 d_model 维度分成 n_heads 个子空间，每个子空间独立计算注意力，
        最后拼接并投影回原始维度。

    数学原理:
        MultiHead(Q, K, V) = Concat(head_1, ..., head_h) @ W_O
        head_i = Attention(Q @ W_Q_i, K @ W_K_i, V @ W_V_i)

    复杂度:
        - 时间: O(n^2 * d) 其中 n 为序列长度，d 为维度
        - 空间: O(n^2 * h) 用于存储 h 个头的注意力矩阵
    """

    def __init__(
        self,
        d_model: int = 512,
        n_heads: int = 8,
        dropout: float = 0.0,
        bias: bool = True,
    ) -> None:
        super().__init__()
        assert (
            d_model % n_heads == 0
        ), f"d_model ({d_model}) must be divisible by n_heads ({n_heads})"

        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads
        self.scale = 1.0 / math.sqrt(self.d_k)

        # 合并的 QKV 投影 (更高效)
        self.W_qkv = nn.Linear(d_model, 3 * d_model, bias=bias)
        self.W_o = nn.Linear(d_model, d_model, bias=bias)
        self.dropout = nn.Dropout(dropout)

    def forward(
        self,
        x: Tensor,
        mask: Optional[Tensor] = None,
        return_attention: bool = False,
    ) -> Tuple[Tensor, Optional[Tensor]]:
        """前向传播。

        Args:
            x: 输入张量 (batch, seq_len, d_model)
            mask: 掩码张量 (batch, 1, seq_len, seq_len) 或 (batch, seq_len, seq_len)
            return_attention: 是否返回注意力权重

        Returns:
            output: 输出张量 (batch, seq_len, d_model)
            attention_weights: 可选的注意力权重 (batch, n_heads, seq_len, seq_len)
        """
        B, T, C = x.shape

        # 合并投影并分割 QKV
        qkv = self.W_qkv(x)  # (B, T, 3*d_model)
        qkv = qkv.reshape(B, T, 3, self.n_heads, self.d_k)
        qkv = qkv.permute(2, 0, 3, 1, 4)  # (3, B, n_heads, T, d_k)
        q, k, v = qkv[0], qkv[1], qkv[2]

        # 注意力分数: einsum 'b h i d, b h j d -> b h i j'
        scores = torch.einsum("b h i d, b h j d -> b h i j", q, k) * self.scale

        # 应用掩码
        if mask is not None:
            if mask.dim() == 3:
                mask = mask.unsqueeze(1)  # (B, 1, T, T)
            scores = scores.masked_fill(mask.bool(), -1e9)

        # Softmax + Dropout
        attn_weights = F.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # 加权求和: einsum 'b h i j, b h j d -> b h i d'
        out = torch.einsum("b h i j, b h j d -> b h i d", attn_weights, v)

        # 合并多头: (B, n_heads, T, d_k) -> (B, T, d_model)
        out = out.transpose(1, 2).contiguous().view(B, T, C)

        # 输出投影
        output = self.W_o(out)

        if return_attention:
            return output, attn_weights
        return output, None


# 测试多头注意力
def test_multi_head_attention() -> None:
    mha = MultiHeadAttention(d_model=256, n_heads=8)
    x = torch.randn(2, 10, 256)
    out, attn = mha(x, return_attention=True)

    print(f"输入形状: {x.shape}")
    print(f"输出形状: {out.shape}")
    print(f"注意力形状: {attn.shape}")
    print(f"参数量: {sum(p.numel() for p in mha.parameters()):,}")


test_multi_head_attention()

---

## 7. Flash Attention 原理与简化实现 ⭐⭐

### 7.1 标准注意力的内存瓶颈

**问题**: 标准注意力需要存储完整的 $n \times n$ 注意力矩阵。

- 序列长度 $n = 4096$, 头数 $h = 32$, batch $B = 8$
- 注意力矩阵大小: $B \times h \times n \times n = 8 \times 32 \times 4096 \times 4096 \approx 17$ GB (FP32)

### 7.2 Flash Attention 核心思想

**关键洞察**: 利用 GPU 内存层次结构，分块计算注意力。

1. **分块 (Tiling)**: 将 Q, K, V 分成小块，每次只处理一块
2. **重计算 (Recomputation)**: 反向传播时重新计算注意力，而非存储
3. **在线 Softmax**: 使用数值稳定的在线算法计算 Softmax

### 7.3 在线 Softmax 算法

标准 Softmax 需要两次遍历:
1. 第一遍: 找最大值 $m = \max(x)$
2. 第二遍: 计算 $\exp(x - m) / \sum \exp(x - m)$

**在线算法** (单次遍历):
$$m_{new} = \max(m_{old}, x_i)$$
$$d_{new} = d_{old} \cdot e^{m_{old} - m_{new}} + e^{x_i - m_{new}}$$

In [ ]:
def flash_attention_simplified(
    q: Tensor,
    k: Tensor,
    v: Tensor,
    block_size: int = 64,
) -> Tensor:
    """Flash Attention 简化实现 (教学版本)。

    注意: 这是简化的 Python 实现，用于理解原理。
    实际 Flash Attention 使用 CUDA kernel 实现，速度快 2-4x。

    核心思想:
        1. 分块处理 K, V，避免存储完整注意力矩阵
        2. 使用在线 Softmax 算法，数值稳定
        3. 累积输出，最后归一化

    Args:
        q: Query (batch, seq_q, d_k)
        k: Key (batch, seq_k, d_k)
        v: Value (batch, seq_k, d_v)
        block_size: 分块大小

    Returns:
        output: (batch, seq_q, d_v)
    """
    B, N, d_k = q.shape
    _, M, d_v = v.shape
    scale = 1.0 / math.sqrt(d_k)

    # 初始化输出和归一化因子
    output = torch.zeros(B, N, d_v, device=q.device, dtype=q.dtype)
    row_max = torch.full((B, N, 1), float("-inf"), device=q.device, dtype=q.dtype)
    row_sum = torch.zeros(B, N, 1, device=q.device, dtype=q.dtype)

    # 分块遍历 K, V
    for j in range(0, M, block_size):
        j_end = min(j + block_size, M)
        k_block = k[:, j:j_end, :]  # (B, block, d_k)
        v_block = v[:, j:j_end, :]  # (B, block, d_v)

        # 计算当前块的注意力分数
        scores = torch.einsum("b i d, b j d -> b i j", q, k_block) * scale  # (B, N, block)

        # 在线 Softmax: 更新最大值
        block_max = scores.max(dim=-1, keepdim=True).values  # (B, N, 1)
        new_max = torch.maximum(row_max, block_max)

        # 重新缩放之前的累积值
        exp_diff = torch.exp(row_max - new_max)
        output = output * exp_diff
        row_sum = row_sum * exp_diff

        # 计算当前块的贡献
        exp_scores = torch.exp(scores - new_max)  # (B, N, block)
        block_sum = exp_scores.sum(dim=-1, keepdim=True)  # (B, N, 1)

        # 累积输出和归一化因子
        output = output + torch.einsum("b i j, b j d -> b i d", exp_scores, v_block)
        row_sum = row_sum + block_sum
        row_max = new_max

    # 最终归一化
    output = output / row_sum

    return output


# 验证 Flash Attention 与标准实现的一致性
def verify_flash_attention() -> None:
    torch.manual_seed(42)
    B, N, d = 2, 128, 64

    q = torch.randn(B, N, d)
    k = torch.randn(B, N, d)
    v = torch.randn(B, N, d)

    # 标准注意力
    config = AttentionConfig(d_k=d, d_v=d)
    std_attn = ScaledDotProductAttention(config)
    std_out, _ = std_attn(q, k, v)

    # Flash Attention
    flash_out = flash_attention_simplified(q, k, v, block_size=32)

    # 比较
    diff = (std_out - flash_out).abs().max().item()
    print(f"标准注意力 vs Flash Attention 最大差异: {diff:.2e}")
    print(f"验证通过: {diff < 1e-5}")


verify_flash_attention()

---

## 8. 复杂度分析与优化策略

### 8.1 标准注意力复杂度

| 操作 | 时间复杂度 | 空间复杂度 |
|:-----|:----------|:-----------|
| QK^T 计算 | $O(n^2 d)$ | $O(n^2)$ |
| Softmax | $O(n^2)$ | $O(n^2)$ |
| Attention @ V | $O(n^2 d)$ | $O(nd)$ |
| **总计** | $O(n^2 d)$ | $O(n^2)$ |

### 8.2 长序列优化方法

| 方法 | 时间复杂度 | 核心思想 |
|:-----|:----------|:---------|
| **Flash Attention** | $O(n^2 d)$ | 分块计算，减少内存 |
| **Sparse Attention** | $O(n \sqrt{n})$ | 稀疏注意力模式 |
| **Linear Attention** | $O(nd^2)$ | 核方法近似 |
| **Sliding Window** | $O(nwd)$ | 局部窗口注意力 |

In [ ]:
def analyze_attention_complexity() -> None:
    """分析不同序列长度下的注意力复杂度。"""
    seq_lengths = [128, 256, 512, 1024, 2048, 4096, 8192]
    d_model = 512
    n_heads = 8
    batch_size = 1

    print("注意力机制复杂度分析")
    print("=" * 70)
    print(f"{'序列长度':<12} {'注意力矩阵大小':<20} {'内存 (FP32)':<15} {'FLOPs':<15}")
    print("-" * 70)

    for n in seq_lengths:
        # 注意力矩阵大小: B * h * n * n
        attn_size = batch_size * n_heads * n * n
        memory_bytes = attn_size * 4  # FP32 = 4 bytes
        memory_mb = memory_bytes / (1024 * 1024)

        # FLOPs: 2 * B * h * n^2 * d_k (QK^T) + 2 * B * h * n^2 * d_k (Attn @ V)
        d_k = d_model // n_heads
        flops = 4 * batch_size * n_heads * n * n * d_k
        gflops = flops / 1e9

        print(f"{n:<12} {attn_size:<20,} {memory_mb:<15.2f} MB {gflops:<15.2f} GFLOPs")

    print("=" * 70)
    print("观察: 内存和计算量随序列长度呈 O(n^2) 增长")


analyze_attention_complexity()

---

## 9. 总结

| 要点 | 说明 |
|:-----|:-----|
| **缩放因子** | $1/\sqrt{d_k}$ 将点积方差从 $d_k$ 归一化为 1 |
| **数值稳定性** | 掩码使用 `-1e9` 而非 `-inf` |
| **einsum** | 提升矩阵运算的数学可读性 |
| **因果掩码** | 防止自回归模型的信息泄漏 |
| **多头注意力** | 并行学习多种关注模式 |
| **Flash Attention** | 分块计算，内存效率提升 2-4x |
| **复杂度** | 时间 $O(n^2 d)$，空间 $O(n^2)$ |

**进阶学习**: RoPE 位置编码、ALiBi、Grouped Query Attention (GQA)